In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet import preprocess_input, MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import Precision, Recall


### 1. Load the data

In [2]:
filename = 'metadata_filtered.csv'

df = pd.read_csv(os.path.join("data", filename))

FileNotFoundError: [Errno 2] No such file or directory: 'data\\metadata_filtered.csv'

In [ ]:
# Benign = 0, Malignant = 1
df['target'] = df['benign_malignant'].map({'benign': 0, 'malignant': 1})
df.head(3)

,Unnamed: 0,image_name,patient_id,lesion_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,2,ISIC_0052212,IP_2842074,IL_9087444,female,50.0,lower extremity,nevus,benign,0
1,3,ISIC_0068279,IP_6890425,IL_4255399,female,45.0,head/neck,unknown,benign,0
2,6,ISIC_0074542,IP_4698288,IL_5017890,male,25.0,lower extremity,unknown,benign,0


In [ ]:
base_model = MobileNet(weights='imagenet', include_top=False, pooling='avg')
model_feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

/var/folders/bt/spx_pkps3xs3hctqcx6dj9kw0000gn/T/ipykernel_4090/2802653591.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(weights='imagenet', include_top=False, pooling='avg')


In [ ]:
def load_and_preprocess_image(image_path):
    """
    Load an image, convert it to an array, preprocess it, and extract features using ResNet50.
    """
    # Adjust the path according to your dataset structure
    full_path = os.path.join('data', 'train', image_path + '.jpg')
    
    # Load and preprocess the image
    img = load_img(full_path, target_size=(224, 224))  # ResNet50 expects input size of 224x224
    img_array = img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(img_array_expanded)
    
    # Extract features using the ResNet50 model
    features = model_feature_extractor.predict(preprocessed_img)
    
    return np.squeeze(features)

### Find patient records with the same body location present in multiple appointments

In [ ]:
unique_patients = df['patient_id'].unique()

In [ ]:
# Lets find series of images for each patient and body area. 1 array for each time step
preprocessing_train_arr = []
preprocessing_label_arr = []

# Iterate over each unique patient
for patient_id in unique_patients:
    patient_df = df[df['patient_id'] == patient_id]
    
    # Iterate over each unique body area for the current patient
    for anatom_site in patient_df['anatom_site_general_challenge'].unique():
        anatom_site_df = patient_df[patient_df['anatom_site_general_challenge'] == anatom_site].copy()
        
        # Sort by 'age_approx' to maintain temporal order. Sort by target to ensure if melanoma is present its in our array.
        anatom_site_df.sort_values(by=['age_approx', 'target'], ascending=[True, False], inplace=True)
        first_images_by_age = anatom_site_df.groupby('age_approx').first().reset_index()
        # Check if there are multiple ages for the current body area
        if len(first_images_by_age) > 1:
            sub_arr = []
            sub_labels = []

            # Keep first N rows:
            # Dataset has 1-4, we've excluded the 1's and will only keep first 2 to keep our dataset homogenous
            rows_to_keep = 2
            first_two_rows = first_images_by_age.head(rows_to_keep)

            # Iterate over each row in the sorted DataFrame
            for _, row in first_two_rows.iterrows():
                # load the image and the image and its label for each age to the sub-lists
                # sub_arr.append(load_and_preprocess_image(row['image_name']))
                sub_arr.append(row['image_name'])
                sub_labels.append(row['target'])
                
            # After processing all ages for the current anatom site, append the sub-lists to the main lists
            preprocessing_train_arr.append(sub_arr)
            preprocessing_label_arr.append(sub_labels)


In [ ]:
print(f"Number of training sequences: {len(preprocessing_label_arr)}")
# number of arrays in label_arr with a 1 in them
print(f"Number of training sequences with melanoma: {sum([1 in labels for labels in preprocessing_label_arr])}")

Number of training sequences: 1801
Number of training sequences with melanoma: 185


In [ ]:
def balance(train_arr, label_arr):
    train_arr = np.array(train_arr)
    label_arr = np.array(label_arr)

    melanoma_indices = [index for index, labels in enumerate(label_arr) if labels[-1] == 1]
    non_melanoma_indices = [index for index, labels in enumerate(label_arr) if 1 not in labels]

    # Balance 60/40. Find count for 60% of melanoma images
    non_melanoma_count = int(len(melanoma_indices) / 0.6 * 0.4)
        
    np.random.seed(0)  # Seed for reproducibility
    selected_non_melanoma_indices = np.random.choice(non_melanoma_indices, len(melanoma_indices), replace=False)

    selected_indices = np.concatenate((melanoma_indices, selected_non_melanoma_indices))
    np.random.shuffle(selected_indices) 

    balanced_train_arr = train_arr[selected_indices]
    balanced_label_arr = label_arr[selected_indices]

    return balanced_train_arr, balanced_label_arr


In [ ]:
preprocessing_train_arr, preprocessing_label_arr = balance(preprocessing_train_arr, preprocessing_label_arr)

In [ ]:
train_arr = [[load_and_preprocess_image(image_name) for image_name in image_sequence] for image_sequence in preprocessing_train_arr]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━

In [ ]:

# Pad sequences to ensure uniform length
X = pad_sequences(train_arr, padding='post', dtype='float32', maxlen=2)

y = np.array([seq[-1] for seq in preprocessing_label_arr]) 


In [ ]:
number_of_sequences_with_melanoma = np.sum(y == 1)
print(f"Number of training sequences with melanoma: {number_of_sequences_with_melanoma}")

Number of training sequences with melanoma: 86


In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 120
Validation set size: 26
Test set size: 26


In [ ]:
model = Sequential([
    Masking(mask_value=0., input_shape=(X.shape[1], X.shape[2])),  
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')  
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])
model.summary()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 2, 1024)           │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ lstm (LSTM)                     │ (None, 64)                │    278,784 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 1)                 │         65 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 278,849 (1.06 MB)

 Trainable params: 278,849 (1.06 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.4823 - loss: 0.6827 - precision: 0.5164 - recall: 0.6514 - val_accuracy: 0.5769 - val_loss: 0.7059 - val_precision: 1.0000 - val_recall: 0.2143
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7312 - loss: 0.5243 - precision: 0.8231 - recall: 0.5786 - val_accuracy: 0.5000 - val_loss: 0.6052 - val_precision: 0.5238 - val_recall: 0.7857
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7750 - loss: 0.4639 - precision: 0.7063 - recall: 0.9937 - val_accuracy: 0.6923 - val_loss: 0.5955 - val_precision: 1.0000 - val_recall: 0.4286
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7740 - loss: 0.4243 - precision: 0.8342 - recall: 0.7107 - val_accuracy: 0.8077 - val_loss: 0.5693 - val_precision: 1.0000 - val_recall: 0.6429
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9156 - loss: 0.3041 - precision: 0.8723 - recall: 0.9768 - val_accuracy: 0.6538 - val_loss: 0.5050 - val_precisio

In [ ]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 2, 1024)           │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ lstm (LSTM)                     │ (None, 64)                │    278,784 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 1)                 │         65 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 836,549 (3.19 MB)

 Trainable params: 278,849 (1.06 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 557,700 (2.13 MB)

In [ ]:
results = model.evaluate(X_val, y_val)

print(f"Validation Loss: {results[0]}")
print(f"Validation Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

results = model.evaluate(X_test, y_test)

print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6154 - loss: 0.9489 - precision: 0.4286 - recall: 0.7500
Validation Loss: 0.9488916397094727
Validation Accuracy: 0.6153846383094788
Test Precision: 0.4285714328289032
Test Recall: 0.75
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7308 - loss: 0.4823 - precision: 0.7692 - recall: 0.7143
Test Loss: 0.48230040073394775
Test Accuracy: 0.7307692170143127
Test Precision: 0.7692307829856873
Test Recall: 0.7142857313156128


In [ ]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.0990 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.6923 - val_loss: 0.5095 - val_precision: 0.7500 - val_recall: 0.6429
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0712 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.7308 - val_loss: 0.5102 - val_precision: 0.7692 - val_recall: 0.7143
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0540 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.7308 - val_loss: 0.5391 - val_precision: 0.7692 - val_recall: 0.7143
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0408 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.6923 - val_loss: 0.6023 - val_precision: 0.7500 - val_recall: 0.6429
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0387 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.6923 - val_loss: 0.5843 - val_precisio

In [ ]:
results = model.evaluate(X_val, y_val)

print(f"Validation Loss: {results[0]}")
print(f"Validation Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

results = model.evaluate(X_test, y_test)

print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")
print(f"Test Precision: {results[2]}")
print(f"Test Recall: {results[3]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5769 - loss: 1.8496 - precision: 0.3846 - recall: 0.6250
Validation Loss: 1.8495597839355469
Validation Accuracy: 0.5769230723381042
Test Precision: 0.38461539149284363
Test Recall: 0.625
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7692 - loss: 0.9572 - precision: 0.7857 - recall: 0.7857
Test Loss: 0.9571620225906372
Test Accuracy: 0.7692307829856873
Test Precision: 0.7857142686843872
Test Recall: 0.7857142686843872


Avoid overtraining to prevent overfitting. 
Precision and recall are better indicators here of success in melanoma detection.